In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.12:3.0.0-beta --conf spark.cassandra.connection.host=127.0.0.1 pyspark-shell'

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
sc = SparkContext('local')
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [3]:
def load_and_get_table_df(keys_space_name, table_name):
    table_df = sqlContext.read\
        .format("org.apache.spark.sql.cassandra")\
        .options(table=table_name, keyspace=keys_space_name)\
        .load()
    return table_df

In [7]:
spacecraft_journey_catalog = load_and_get_table_df("demo", "spacecraft_journey_catalog")
spacecraft_journey_catalog.show()
spacecraft_journey_catalog.createTempView('spacecraft_journey_catalog')

+---------------+--------------------+------+-------------------+-------------------+--------------------+
|spacecraft_name|          journey_id|active|                end|              start|             summary|
+---------------+--------------------+------+-------------------+-------------------+--------------------+
|         sts-64|73f5f400-cc24-11c...| false|1994-09-20 17:11:00|1994-09-09 18:22:00|Atmospheric and s...|
|        soyuz13|b1a20200-8f44-11b...| false|1973-12-26 03:50:00|1973-12-18 06:55:00|Second test of re...|
|      soyuztm-5|2c6dc200-c84f-11c...| false|1988-06-17 06:13:00|1988-06-07 10:03:00|                    |
|      datastax7|b0a04f00-5d8d-11e...| false|2021-03-09 08:22:55|2021-01-05 22:27:37|        Land on Mars|
|      datastax7|54564900-54ca-11e...| false|2021-06-13 09:39:39|2021-03-08 08:54:23|Find new forms of...|
|      datastax7|d3cd1500-5403-11e...| false|2021-12-17 06:38:20|2021-03-16 21:07:29|        Land on Mars|
|      datastax7|1210f880-5360-11e...

In [8]:
spacecraft_speed_over_time = load_and_get_table_df("demo", "spacecraft_speed_over_time")
spacecraft_speed_over_time.show()
spacecraft_speed_over_time.createTempView('spacecraft_speed_over_time')

+---------------+--------------------+-------------------+-------+----------+
|spacecraft_name|          journey_id|       reading_time|  speed|speed_unit|
+---------------+--------------------+-------------------+-------+----------+
|     datastax33|0c9d5500-1058-11f...|1969-07-16 09:32:17|30077.0|      km/h|
|     datastax33|0c9d5500-1058-11f...|1969-07-16 09:32:16|30076.0|      km/h|
|         soyuz6|16b2f400-ddc0-11b...|1969-07-16 09:32:07|30035.0|      km/h|
|         soyuz6|16b2f400-ddc0-11b...|1969-07-16 09:32:06|30033.0|      km/h|
|        vostok1|805b1a00-5673-11a...|1969-07-16 09:32:01|30004.0|      km/h|
|        vostok1|805b1a00-5673-11a...|1969-07-16 09:32:00|30000.0|      km/h|
|     datastax21|a0ffe880-7b2b-11e...|1969-07-16 09:32:15|30064.0|      km/h|
|     datastax21|a0ffe880-7b2b-11e...|1969-07-16 09:32:14|30062.0|      km/h|
|         sts-37|15c08000-f345-11c...|1969-07-16 09:32:09|30041.0|      km/h|
|         sts-37|15c08000-f345-11c...|1969-07-16 09:32:08|30039.

In [9]:
spark.sql("select spacecraft_journey_catalog.spacecraft_name, spacecraft_journey_catalog.summary, spacecraft_speed_over_time.speed from spacecraft_journey_catalog inner join spacecraft_speed_over_time on spacecraft_journey_catalog.journey_id = spacecraft_speed_over_time.journey_id").show()

+---------------+--------------------+-------+
|spacecraft_name|             summary|  speed|
+---------------+--------------------+-------+
|      datastax5|Bring passengers ...|30060.0|
|      datastax5|Bring passengers ...|30059.0|
|     datastax33|       Crew rotation|30077.0|
|     datastax33|       Crew rotation|30076.0|
|        sts-101|ISS construction ...|30052.0|
|        sts-101|ISS construction ...|30043.0|
|     datastax21|Bring supplies to...|30064.0|
|     datastax21|Bring supplies to...|30062.0|
|         soyuz6|First three-craft...|30035.0|
|         soyuz6|First three-craft...|30033.0|
|         sts-37|Deployment of Com...|30041.0|
|         sts-37|Deployment of Com...|30039.0|
|        gemini7|First two-week sp...|30022.0|
|        gemini7|First two-week sp...|30021.0|
| mercury-atlas6|First American ma...|30007.0|
| mercury-atlas6|First American ma...|30006.0|
|        vostok1|First manned spac...|30004.0|
|        vostok1|First manned spac...|30000.0|
+------------